In [1]:
from rate_movies_helper import posters, dataset

In [2]:
from IPython.display import Image
import pandas as pd

In [3]:
x = posters.get_finished_movie_ids()

In [4]:
Image(url=x.loc[588, 'poster'])

In [5]:
ratings = dataset.load('ratings.csv').drop('timestamp', axis=1)

In [6]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


# 

In [22]:
def get_movieId_vs_n_ratings(ratings_df):
    ratings_df['n_ratings'] = 1
    return ratings[['n_ratings', 'movieId']].groupby('movieId').count()

In [ ]:
def get_most_rated(ratings_df, top_n):
    

In [24]:
get_movieId_vs_n_ratings(ratings).head()

,n_ratings
movieId,
1,215
2,110
3,52
4,7
5,49


In [9]:
import pandas as pd